### Importing the libraries

- numpy for numerical computations;
- pandas for data analysis;
- matplotlib for plotting;

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Features and dependent variables

In any dataset where you want to train a machine learning model, you have the same entities: features and the dependent variables

- Features - Columns where you want to predict the dependent variable;
- Dependent variable - Column where you will use to train the model to predict them.

In [14]:
dataset = pd.read_csv('Data.csv')

features = dataset.iloc[:, :-1].values
dependent = dataset.iloc[:, -1].values

print(features)
print(dependent)


[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]
['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


### Taking care of the missing data

In [16]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

imputer.fit(features[:, 1:])
features[:, 1:] = imputer.transform(features[:, 1:])

features

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, 63777.77777777778],
       ['France', 35.0, 58000.0],
       ['Spain', 38.77777777777778, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)